In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv(r"C:\Users\Komal\Downloads\covid_toy.csv")

In [4]:
from sklearn.preprocessing import StandardScaler

In [17]:
sc = StandardScaler()
sc.fit(df[['fever']].values)
_=sc.transform([[89]])

In [18]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [19]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [20]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [21]:
X_train

,age,gender,fever,cough,city
71,75,Female,104.0,Strong,Delhi
85,16,Female,103.0,Mild,Bangalore
25,23,Male,NaN,Mild,Mumbai
3,31,Female,98.0,Mild,Kolkata
91,38,Male,NaN,Mild,Delhi
...,...,...,...,...,...
19,42,Female,NaN,Strong,Bangalore
67,65,Male,99.0,Mild,Bangalore
38,49,Female,101.0,Mild,Delhi
93,27,Male,100.0,Mild,Kolkata


In [22]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

In [23]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [24]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [25]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [26]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

In [27]:
from sklearn.compose import ColumnTransformer

In [31]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [32]:
transformer.fit_transform(X_train).shape

(80, 7)

In [33]:
transformer.transform(X_test).shape

(20, 7)

In [34]:
transformer1 = ColumnTransformer(transformers =[
    ('imputer',SimpleImputer(),['fever']),
    ('ordinal',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('onehot',OneHotEncoder(drop = 'first',sparse_output = False),['gender','city'])
], remainder = 'passthrough')

In [35]:
transformer1.fit(X_train)

C:\Users\Komal\miniconda3\Lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


ColumnTransformer(remainder='passthrough',
                  transformers=[('imputer', SimpleImputer(), ['fever']),
                                ('ordinal',
                                 OrdinalEncoder(categories=[['Mild',
                                                             'Strong']]),
                                 ['cough']),
                                ('onehot',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['gender', 'city'])])

In [36]:
transformer1.transform(X_test)

array([[ 98.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  83.        ],
       [ 98.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  24.        ],
       [100.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  13.        ],
       [103.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  73.        ],
       [103.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  50.        ],
       [103.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  69.        ],
       [100.77777778,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  82.        ],
       [100.77777778,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  20.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          0.    

In [37]:
X_test

,age,gender,fever,cough,city
53,83,Male,98.0,Mild,Delhi
82,24,Male,98.0,Mild,Kolkata
24,13,Female,100.0,Strong,Kolkata
69,73,Female,103.0,Mild,Delhi
39,50,Female,103.0,Mild,Kolkata
16,69,Female,103.0,Mild,Kolkata
41,82,Male,NaN,Mild,Kolkata
7,20,Female,NaN,Strong,Mumbai
13,64,Male,102.0,Mild,Bangalore
22,71,Female,98.0,Strong,Kolkata


In [28]:
scatterplot(data = df, x = 'cgpa',y = 'lpa')

NameError: name 'scatterplot' is not defined